In [5]:
!unzip /content/pdfs/pdfs_comprimidos.zip

Archive:  /content/pdfs/pdfs_comprimidos.zip
 extracting: L14/BOCG-14-B-108-1.PDF  
 extracting: L14/BOCG-14-B-137-1.PDF  
 extracting: L14/BOCG-14-B-52-1.PDF  
 extracting: L14/BOCG-14-B-65-1.PDF  
 extracting: L14/BOCG-14-B-45-1.PDF  
 extracting: L14/BOCG-14-B-85-1.PDF  
 extracting: L14/BOCG-14-B-78-1.PDF  
 extracting: L14/BOCG-14-B-100-1.PDF  
 extracting: L15/BOCG-15-B-69-1.PDF  
 extracting: L15/BOCG-15-B-32-1.PDF  
 extracting: L15/BOCG-15-B-112-1.PDF  
 extracting: L15/BOCG-15-B-121-1.PDF  
 extracting: L15/BOCG-15-B-43-1.PDF  
 extracting: L15/BOCG-15-B-47-1.PDF  
 extracting: L15/BOCG-15-B-81-1.PDF  
 extracting: L15/BOCG-15-B-59-1.PDF  
 extracting: L13/BOCG-13-B-82-1.PDF  
 extracting: L13/BOCG-13-B-51-1.PDF  
 extracting: L13/BOCG-13-B-62-1.PDF  
 extracting: L13/BOCG-13-B-71-1.PDF  
 extracting: L13/BOCG-13-B-40-1.PDF  
 extracting: L12/BOCG-12-B-100-1.PDF  
 extracting: L12/BOCG-12-B-128-1.PDF  
 extracting: L12/BOCG-12-B-20-1.PDF  
 extracting: L12/BOCG-12-B-65-1.PDF 

In [6]:
!pip install pymupdf


  Using cached PyMuPDF-1.24.7-cp310-none-manylinux2014_x86_64.whl (3.5 MB)
  Using cached PyMuPDFb-1.24.6-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (15.7 MB)


In [24]:
import pandas as pd
import fitz  # PyMuPDF para extraer texto de PDFs
import re
from nltk.corpus import stopwords

# Descargar stopwords si no están descargadas
import nltk
nltk.download('stopwords')

# Función para extraer texto de un archivo PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Función para preprocesar el texto y eliminar las 20 primeras y últimas palabras
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Eliminar números
    text = re.sub(r'\W+', ' ', text)  # Eliminar caracteres especiales
    words = text.split()
    words = [word for word in words if word not in stopwords.words('spanish')]  # Eliminar stopwords

    # Eliminar las 20 primeras y 20 últimas palabras
    if len(words) > 40:  # Asegurarse de que haya al menos 40 palabras
        words = words[20:-20]
    else:
        words = []  # Si no hay suficientes palabras, se deja vacío

    return ' '.join(words)


# Cargar el dataset desde Excel
dataset = pd.read_excel('/content/DatasetL11-L15.xlsx')

# Crear listas para almacenar los textos preprocesados
textos_economico = []
textos_social = []

# Iterar sobre cada fila del dataset
for index, row in dataset.iterrows():
    ruta_pdf = row['Ruta']
    try:
        # Extraer texto del PDF
        texto_pdf = extract_text_from_pdf(ruta_pdf)
        # Preprocesar el texto
        texto_procesado = preprocess_text(texto_pdf)
    except Exception as e:
        print(f"Error al procesar PDF en ruta: {ruta_pdf}. Error: {str(e)}")
        texto_procesado = ''  # Agregar cadena vacía en caso de error

    # Verificar el tipo y agregar el texto procesado a la lista correspondiente
    if row['Economico']:
        textos_economico.append(texto_procesado)
    else:
        textos_economico.append('')

    if row['Social']:
        textos_social.append(texto_procesado)
    else:
        textos_social.append('')

# Agregar las listas de textos preprocesados al dataset como nuevas columnas
dataset['Texto_Economico'] = textos_economico
dataset['Texto_Social'] = textos_social

# Crear nuevos datasets solo con las columnas necesarias
dataset_economico = dataset[['Economico', 'Ruta', 'Texto_Economico']]
dataset_social = dataset[['Social', 'Ruta', 'Texto_Social']]

# Guardar los nuevos datasets actualizados
dataset_economico.to_excel('dataset_economico.xlsx', index=False)
dataset_social.to_excel('dataset_social.xlsx', index=False)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [32]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib
import random
random_number = random.randint(1, 100)

# Cargar el dataset con texto preprocesado desde Excel
dataset = pd.read_excel('/content/dataset_economico.xlsx')

# Eliminar filas con NaN en la columna 'Texto'
dataset = dataset.dropna(subset=['Texto_Economico'])

# Dividir los datos en entrenamiento y prueba
X = dataset['Texto_Economico']
y = dataset['Economico']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=47)
print(f'Semilla aleatoria utilizada en train_test_split: 47')

# Crear un objeto CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 2))  # Utilizar n-grams de 1 y 2 palabras

# Ajustar y transformar los datos de entrenamiento
X_train_vec = vectorizer.fit_transform(X_train)

# Transformar los datos de prueba
X_test_vec = vectorizer.transform(X_test)

# Inicializar y entrenar un modelo de Regresión Logística
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

# Predecir sobre los datos de prueba
y_pred = model.predict(X_test_vec)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Exactitud del modelo: {accuracy:.2f}')

# Imprimir el reporte de clasificación
print(classification_report(y_test, y_pred))

# Guardar el modelo entrenado para su uso posterior
joblib.dump(model, 'modelo_bag_of_ngrams_economico_texto.pkl')

Semilla aleatoria utilizada en train_test_split: 47
Exactitud del modelo: 1.00
              precision    recall  f1-score   support

          -1       1.00      1.00      1.00         4

    accuracy                           1.00         4
   macro avg       1.00      1.00      1.00         4
weighted avg       1.00      1.00      1.00         4



['modelo_bag_of_ngrams_economico_texto.pkl']

In [36]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import joblib
import random

random_number = random.randint(1, 100)

# Cargar el dataset con texto preprocesado desde Excel
dataset = pd.read_excel('/content/dataset_social.xlsx')

# Eliminar filas con NaN en la columna 'Texto'
dataset = dataset.dropna(subset=['Texto_Social'])

# Dividir los datos en entrenamiento y prueba
X = dataset['Texto_Social']
y = dataset['Social']  # Cambiar a la columna 'Social' como variable objetivo
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=83)
print(f'Semilla aleatoria utilizada en train_test_split: 83')

# Crear un objeto CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 2))  # Utilizar n-grams de 1 y 2 palabras

# Ajustar y transformar los datos de entrenamiento
X_train_vec = vectorizer.fit_transform(X_train)

# Transformar los datos de prueba
X_test_vec = vectorizer.transform(X_test)

# Inicializar y entrenar un modelo de Regresión Logística
model = LogisticRegression(max_iter=1000)
model.fit(X_train_vec, y_train)

# Predecir sobre los datos de prueba
y_pred = model.predict(X_test_vec)

# Evaluar el rendimiento del modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Exactitud del modelo: {accuracy:.2f}')

# Imprimir el reporte de clasificación
print(classification_report(y_test, y_pred, zero_division=1))  # Establecer zero_division=1 para evitar el warning

# Guardar el modelo entrenado para su uso posterior
joblib.dump(model, 'modelo_bag_of_ngrams_social_texto.pkl')


Semilla aleatoria utilizada en train_test_split: 83
Exactitud del modelo: 0.71
              precision    recall  f1-score   support

          -2       1.00      0.00      0.00         1
           2       1.00      0.00      0.00         1
           3       0.80      1.00      0.89         4
           4       0.50      1.00      0.67         1

    accuracy                           0.71         7
   macro avg       0.82      0.50      0.39         7
weighted avg       0.81      0.71      0.60         7



['modelo_bag_of_ngrams_social_texto.pkl']

In [ ]:
import fitz  # PyMuPDF para extraer texto de PDFs
import re
import joblib

# Función para extraer texto de un archivo PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text()
    return text

# Función para preprocesar el texto
def preprocess_text(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)  # Eliminar números
    text = re.sub(r'\W+', ' ', text)  # Eliminar caracteres especiales
    words = text.split()
    words = [word for word in words if word not in stopwords.words('spanish')]  # Eliminar stopwords
    return ' '.join(words)

# Cargar el modelo entrenado
modelo_path = 'modelo_bag_of_ngrams_economico_texto.pkl'
model = joblib.load(modelo_path)

# Ruta del PDF que deseas procesar
ruta_pdf = '/content/pdfs/L11/BOCG-11-B-18-1.PDF'

# Extraer texto del PDF
try:
    texto_pdf = extract_text_from_pdf(ruta_pdf)
    # Preprocesar el texto
    texto_procesado = preprocess_text(texto_pdf)
    # Vectorizar el texto
    texto_vec = vectorizer.transform([texto_procesado])
    # Realizar la predicción
    predicciones = model.predict(texto_vec)
    print(f'Predicción para el PDF {ruta_pdf}: {predicciones}')
except Exception as e:
    print(f"Error al procesar PDF en ruta: {ruta_pdf}. Error: {str(e)}")


Predicción para el PDF /content/pdfs/L11/BOCG-11-B-18-1.PDF: [1]
